# Bioinformática con Python simple

## Contenidos de este tutorial

1. Manejo de secuencias como strings
    1. Búsqueda de substring
    1. Manejo de strings
    1. Secuencia complementaria por reemplazo de bases
    
1. Expresiones regulares
    1. Método search
    1. Método findall
    1. expresiones regulares
    
1. Invocación de archivo Python desde línea de comando
    1. Encabezado del archivo, permisos
    1. Invocación con argumentos

1. Ejemplos
    1. Indexación de k-mers de una secuencia
    1. Parseo de archivos FASTA

1. Tarea
    1. Presentación
    1. Entregables
    1. Pograma 1: Alineamiento de partidores degenerados
    1. Pograma 2: Identificación y transcripción de ORFs
  

## 1.Manejo de secuencias como Strings

La clase string provee algunos métodos que podemos utilizar para manejar secuencias

In [ ]:
ADN = 'CATCTACTAGCTAGCTAGCTATGTAGTAGCCTACTAGCTAGCTAGCATTTCATATCTATCTAG'
print(ADN)

### 1.A Búsqueda de substring

Se puede buscar un substring con el método **find**, que retornará la posición (0-based) de la primera ocurrencia del patrón buscado a partir de la posición indicada (por defecto el principio del string). Si no está presente, retorna -1.


In [ ]:
def busca(ADN:str, seq:str):
    pos = -1
    while True:
        pos = ADN.find(seq, pos+1)
        if pos != -1:
            print(f'secuencia {seq} encontrada en la posición {pos}')
        else:
            break
    
ADN = 'CATCTACTAGCTAGCTAGCTATGTAGTAGCCTACTAGCTAGCTAGCATTTCATATCTATCTAG'
busca(ADN,'TAG')

### 1.B Manejo de strings

Los strings se manejan como listas (0-based), desde las cuales se pueden extraer subtrings, o bien invertirlos


In [ ]:
st = "abcdefgh"

print(st[3:5]) # rango semiabierto: imprime los caracteres en posiciones 3 y 4

print(st[::-1]) # imprime string en orden inverso

### 1.C Secuencia complementaria por reemplazo de bases

Para obtener la secuencia complementaria podemos utilizar una función que tiene un diccionario con la conversión deseada

In [ ]:
def complementario(DNA):
    comp = {'A':'T','T':'A','G':'C','C':'G'}
    DNAc = ''
    for b in DNA:
        DNAc = DNAc + comp[b]
    return DNAc

DNA2 = 'CTAATGT'

print(f'La secuencia complementaria de {DNA2} es {complementario(DNA2)}')

Para obtener la secuencia de la hebra complementaria (desde 5' a 3' de la hebra complementaria), se debe calcular el reverso complementario:

In [ ]:
DNA3 = 'CCCGGT'
print(f'La secuencia reversa complementaria de {DNA3} es {complementario(DNA3)[::-1]}')

## 2. Expresiones regulares

### 2.A Expresiones regulares

En algunas ocasiones es necesario encontrar un texto que responde a un patrón. Por ejemplo, cuando se desea validar una dirección de e-mail, en el formato **prefijo@dominio** en donde existen condiciones talees como que el prefijo no puede comenzar con un número o caracter especial, y el dominio debe terminar con un punto seguido de un dominio existente (.com, .cl, etc.). En este patrón, pueden calzar millones de direcciones de e-mail diferentes, y nos convendría poder verificarlas de una manera genérica.

Las expresiones regulares pueden componerse de distintas formas, al igual como se utilizan en **sed** o **awk**. A continuación se explican algunos de los caracteres especiales para construirlas (ver más detalle [acá](https://docs.python.org/3/library/re.html)).


| caracter | significado | ejemplo | explicación |
| --- | --- | --- | --- |
| . | cualquier caracter único| CA.A | reconoce strings *CASA*, *CAmA*, *CA!A*, *CA A*etc. |
| ^ | inicio del string | ^El | reconoce *solo* strings que comiencen con *El* |
| \$ | fin del string | da$ | reconoce *solo* strings que terminen en *da* |
| ( ) | agrupación | (ABC) | reconoce *solo* strings que contengan *ABC* |
| \[ \] | conjunto de caracteres| \[ABC\] | reconoce strings que tengan A, B o C |
| * | 0 o más caracteres| \[A-Z\]*| reconce strings compuestos por 0 o más letras mayúsculas |
| + | 1 o más caracteres | \[A-Z\]+| reconce strings compuestos por 1 o más letras mayúsculas |
| \ | escape | ción\\.| reconoce strings terminados en *ción.* (con punto final)|
| \| | OR | (AA\|BB) | reconoce strings *AA* o *BB*|



### 2.B Método search

Python provee la librería **re** para el manejo de expresiones regulares. Una de las funciones principales es **search**, que devuelve un objeto de clase re.Match (match object). En el siguiente ejemplo, dado un string, se desea encontrar un substring que cumpla con contener el string **cons**, terminar en **des** y entre ellos tener 0 o mas caracteres cualquiera.

In [ ]:
import re as re

txt = 'El rey de Constantinopla se quiere desconstantinopolizar, el que logre desconstatinopolizarlo, desconstantinopolizador será'

x = re.search('cons.*des',txt)

print('Match object:')
print(type(x))
print(x)


El objeto match contiene distinta información (el string original, las posiciones de match, etc.), las cuales pueden ser obtenidas:

In [ ]:

print('String original:')
print(x.string)


In [ ]:

print('String que hizo match:')
print(x.group())


In [ ]:

print('Coordenadas del string que hizo match:')
print(str(x.span()))

print(txt[x.span()[0]:x.span()[1]])


In [ ]:

print('Coordenada de inicio del string que hizo match:')
print(str(x.start()))


In [ ]:

print('Coordenada de fin del string que hizo match:')
print(str(x.end()))


### 2.C Método findall

El método **search** retorna el primer match que encuentra en el string. Para retornar todos los match posibles, se puede usar **findall**, que retorna una lista con los strings identificados.

Una expresión regular puede hacer match en varias partes del string. Es posible entregar una expresión regular al método **findall** y obtener una lista con todos los strings identificados. El siguiente ejemplo identifica dentro de una cadena de ARNm los posibles codones que pudieran traducirse al aminoácido Serina. La Serina está sintetizada a partir de 6 codones distintos: UCA, UCG, UCC, UCU, AGC y AGU. Estas cuatro combinaciones pueden ser representadas por la expresión regular **(UC.|AG\[CU\])** ("UC" seguido de cualquier letra única, o bien "AG" seguido de "C" o "U")

In [ ]:
RNA1 = 'UCAGU'
print(re.findall('(UC.|AG[CU])',RNA1))

RNA2 = 'UCAGUGGCUAGCCGGUUCACAUUCUACUCG'
print(re.findall('(UC.|AG[CU])',RNA2))


Note que findall **no encuentra expresiones traslapadas**. Para RNA1 sólo encuentra **UCA** y no **AGU**.

## 3. Invocación de archivo Python desde línea de comando

### 3.A Encabezado del archivo, permisos

En Linux, puede hacerse que un programa en Python sea ejecutable indicando al sistema operativo dos cosas:
- Que el programa debe ejecutarse con el intérprete de python
- Que el archivo es ejecutable

Para lo primero es necesario incluir dos líneas al principio del programa, que indiquen al sistema operativo con qué programa (intérprete Pyton) debe ejecutarse el programa, y la codificación del archivo.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

def despedida():
    print('Chao mundo')

def main():
    print('Hola mundo')
    despedida()

if __name__ == "__main__":
    main()
    

la condición **if** se activa si el programa es invocado directamente, ejecutando lo que haya en la función **main**. Por el contrario, si el programa es invocado mediante la cláusula **include** desde otro prorama, **main** no se ejecutará, pero el programa podrá usar la definición de las funciones que están en el archivo, tales como **depedida**.

Para indicar al sistema operativo que este archivo es ejecutable, debemos asignar permiso de ejecución al archivo mediante el comando chmod:

  **$ chmod +x archivo_python.py**
 
De esta forma, el archivo podrá invocarse sin mencionar explícitamente al intérprete de Python de la siguiente forma:

  **$ ./archivo_python.py**

### 3.B Invocación con argumentos

Al invocar un archivo ejecutable, se le pueden pasar parámetros a través de la línea de comandos, los cuales son recibidos en la lista **sys.argv**. El siguiente programa informa sobre los argumentos de invocación y verifica que la cantidad de ellos sea correcto (este ejemplo no funciona al ejecutarlo en Jupyter)

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import sys

def main():
    print(f'Existen {len(sys.argv)} argumentos de invocación y son:')
    print(sys.argv)
    
    # verificación de argumentos de invocación
    if len(sys.argv) != 3:
        print (f'USO: {sys.argv[0]} <archivo FASTA> <archivo BED>')
        sys.exit()
        
if __name__ == "__main__":
    main()

De esta forma, si **programa.py** espera recibir dos parámetros (además del nombre del archivo ejecutable), la respuesta al ser invocado con un número ddiferente de parámetros será

**$ ./programa.py**<br>
Existen 1 argumentos de invocación y son:<br>
\['./programa.py'\]<br>
USO: ./ejecución.py \<archivo FASTA> \<archivo BED>


**$ ./programa.py bla ble**<br>
Existen 3 argumentos de invocación y son:<br>
\['./programa.py', 'bla', 'ble'\]



## 4. Ejemplos

### 4.A Indexación de k-mers de una secuencia

En este ejemplo se creará una función que recibe una secuencia de ADN y un valor k, para retornar un diccionario con el índice de los k-mers encontrados en la secuencia.


In [ ]:
def make_index(DNA, k):
    index = dict() # diccionario en donde se almacenará el índice
    pos = 0
    while pos < len(DNA)-k+1:
        if DNA[pos:pos+k] not in index:
            # crear registro en diccionario
            index[DNA[pos:pos+k]] = [pos]
        else:
            # agregar posición
            index[DNA[pos:pos+k]].append(pos)
        pos += 1
    return index

DNA = 'CATCTCATC'

index = make_index(DNA,3)
print("Indice con k=3: ")
print(index)

index = make_index(DNA,4)
print("Indice con k=4: ")
print(index)


### 4.B Parseo de archivos FASTA

En este ejemplo se creará una función que lee un archivo FASTA y retorna un diccionario cuya **clave** es el encabezado de cada secuencia del FASTA y **valor** la secuencia correspondiente. Hay que tener en cuenta que la secuencia puede estar dividida en distintas líneas y que pueden existir líneas vacías.

Como ejemplo consideraremos el siguiente archivo FASTA:

In [ ]:
!cat Test.fasta

In [ ]:
def parsea_FASTA(archivof):
    records = dict()
    with open(archivof, "r") as f:
        primer_registro = True
        for l in f:
            if l[0] == ">":
                if not primer_registro:
                    records[header] = seq
                header = l[1:].strip()
                seq =""
                primer_registro = False
            else:
                seq = seq+l.strip()
        records[header] = seq
    return records

rec = parsea_FASTA("Test.fasta")
print(rec)

## 5. Tarea 2

### 5.A Presentación

Esta tarea tiene como propósito generar dos utilidades para el procesamiento de secuencias, introduciendo dos conceptos. La correcta realización de esta tarea demostrará su manejo de los conceptos biológicos y su habilidad en programación.

### 5.B Entregables

Se deberán entregar dos archivos Python, los cuales serán subidos a Siveduc en la fecha acordada, comprimidos en un archivo zip con formato de nombre **Bioinfo_Tarea2_nombre_apellido.zip**. Estos dos programas desberán implementar los dos algoritmos descritos a coninuación.


### 5.C Programa 1: Alineamiento de partidores degenerados

Crear un programa invocable desde la línea de comandos que reciba una secuencia de ADN, un partidor degenerado, y retorne todas las posiciones en que éste último se alineará en la secuencia.

**¿Qué es un partidor degenerado?** La realización de PCR requiere del diseño y síntesis de secuencias que se alineen en los extremos del amplicón a amplificar, de tal manera de incluir un punto de partida para la polimerización. Usualmente estos partidores se definen como una secuencia de las 4 bases nitrogenadas (A, C, G, T). No obstante, en algunas ocasiones, estos partidores pueden encontrarse en zonas con mutaciones reportadas (por ejemplo, humanos de un grupo étnico que se caracteriza por poseer una variante), o  bien se desea diseñar partidores que difieran levenente unos de otros. Para estos efectos, se diseñan partidores que tengan más de una posibilidad de base en alguna posición. Para especificar estos partidores se utilizan los códigos IUPAC:

<img src="images/IUPAC-code-is-adapted-from-100.png" alt="IUPAC codes" style="width: 500px;"/>

Por ejemplo, considerando que **S** puede ser G o C, y **K** puede ser G o T el partidor degenerado **GSTAK** codifica los siguientes 4 partidores:
- G**G**TA**G**
- G**G**TA**T**
- G**C**TA**G**
- G**C**TA**T**

El programa debe devolver una salida con 2 columnas, correspondientes a la posición en donde existe coincidencia, y la secuencia (del ADN) en la cual el primer degenerado se alinea, tal como en el siguiente ejemplo:

> **Formato de invocación**<br>
>     $ ./degenerate_primer_alignment.py ACATGTATGATCTGGTGATTTGTAAGA TST<br>
>     pos seq<br>
>     10 TCT<br>
>      3 TGT<br>
>     20 TGT<br>


### 5.D Programa 2: Identificación y transcripción de ORFs

Crear un programa invocable desde la línea de comandos que reciba una secuencia, y retorne todos los ORFS posibles tanto en la hebra positiva como negativa, así como los transcriptos proteicos que éstos generan.

**¿Qué es un ORF?** Un ORF es el acrónimo de *Open Reading Frame*, el cual es un segmento de secuencia capaz de generar un transcripto proteico. Para efectos de esta traea, considerarmos ORF las secuencias que cumplan con las siguientes características:

1. Debe comenzar con un codón de start (AUG)
1. Debe terminar con un codón de stop (UAA, UAG, UGA)
1. Su longitud debe ser múltiplo de 3
1. No debe contener codones de start ni stop (en el mismo marco de lectura) a excepción de los del principio y fin

(recuerde que U en RNAm es el equivalente a T en ADN)

El mapeo desde bases de RNAm a codones se puede obtener usando el siguiente diagrama:

<img src="images/Aminoacids_table.svg.png" alt="Aminoacid traduction table" style="width: 500px;"/>

Desarrollaremos un ejemplo para la secuencia **AGATGCCTATGGATGTTAATGATTGAGTGATATAACG**. Si buscamos la coincidencia de los codones de start y stop, identificaremos lo siguiente:

<img src="images/codones_start_stop.png" alt="codones de start y stop" style="width: 500px;"/>

No obstante, no todas estos codones se encuentran en el mismo marco de lectura. Para identificar los codones, presentaremos los 3 posibles marcos de lectura, partiendo por el primero (0). En este **primer marco de lectura**, el desfase de lectura es cero, lo que significa que le primer codón estará conformado por las tres primeras letras de la secuencia, el segundo por las tres segundas, y así sucesivamente. Los número en gris identifican el número (1-based) del codón en la secuencia.

<img src="images/marco0.png" alt="Aminoacid traduction table" style="width: 500px;"/>

En este marco de lectura aparecen dos codones de start y uno de stop. Dado que la lectura comienza desde el inicio, el único trascripto comienza en el 5° codón y termina en el 10°. El 7° codón solo agrega una Met a la cadena de aminoácidos de la proteína.

Una vez identificado el ORF (**ATGTTAATGATTGAGTGA**), se pueden obtener los aminoácidos que ésta genera, caracterizando la proteína como **MLMIE** (ver letras en diagrama).

El **segundo marco de lectura** se obtiene desplazando una letra desde el comienzo de la secuencia, por lo que el primer codón comenzará en la sugnda posición de la secuencia original. Para el sgundo marco de lectura solo encontramos un codon de stop y ninguno de start, por lo que este marco no generará ninguna proteína.

<img src="images/marco1.png" alt="Aminoacid traduction table" style="width: 500px;"/>

Para el **tercer marco de lectura** se observan dos codones de start y dos de stop. Aplicando el mismo criterio que para el primer marco de lectura, el transcripto comenzará en el primer codón de start y teminará en el primer codón de stop que se encuentre en ese marco. El ORF será **ATGCCTATGGATGTTAATGATTGA**, y su correspondiente proteína **MPMDVND**.

<img src="images/marco2.png" alt="Aminoacid traduction table" style="width: 500px;"/>

Es posible que exista más de un ORF en el mismo marco de lectura, simpre que su intersección sea nula, es decir, que un ORF termine antes que comienze el siguiente.

Hasta ahora se ha aplicado este análisis solo a la hebra positiva. Para verificar si existe algún ORF en la hebra negativa, el análisis debe reralizarse de la misma manera en la hebra complementaria, la que se obtiene calculando el reverso complementario de la cadena original.

El programa debe devolver una salida con 5 columnas, correspondientes al ORF, la posición de incio (0-based), el largo del ORF, la hebra (+ o -) y el transcripto a proteína, tal como en el siguiente ejemplo:

> **Formato de invocación**<br>
>     $ ./ORF_transcript.py GAGCATGCATGCTATTACATGCTA<br>
>     ORF                      pos len  strand  transcript<br>
>     ATGTTAATGATTGAGTGA        11  18       +  MLMIE<br>
>     ATGCCTATGGATGTTAATGATTGA   2  24       +  MPMDVND<br>


